# AFLEs Agile12v2016
Sandy Wong 2022-02-27

### Imports

In [1]:
import pandas as pd
import numpy as np
import csv
import math
import os

### Functions

In [38]:
def getDataFromFcsv(inFile): #inFile = fcsv with annotation data 

    version = 0
    coordSys = ""
    cols = []
    data = []

    with open(inFile, newline='') as csvfile:
        csvreader = csv.reader(csvfile, delimiter='\n')
        for row in csvreader:
            if row[0][0] == "#": #header in fcsv
                rowVals = row[0].split(" ")
                if rowVals[1] == "Markups":
                    version = float(rowVals[-1])
                if rowVals[1] == "CoordinateSystem":
                    coordSys = str(rowVals[-1])
                if rowVals[1] == "columns":
                    cols = rowVals[-1].split(",")
                    
            else:  #data in fcsv
                rawdata = row[0].split(",")
                data.append(rawdata)
    
    #print(data)
    df = pd.DataFrame(data, columns = cols )
    
    data_dict = {"version":version,
                 "coordinateSystem": coordSys,
                 "data":df}
    
    return data_dict

### Loop through filenames and get data 

#### Filenames

In [39]:
path_Agile12v2016_RATER = "C:\\Users\\Sandy\\Documents\\AFIDs-Data\\Agile12v2016-RATER"

files_Agile12v2016_RATER = os.listdir(path_Agile12v2016_RATER)

In [47]:
Agile12v2016_RATER = []
col_headers = ["id","x","y","z","ow","ox","oy","oz","vis","sel","lock","label","desc","associatedNodeID"]
for file in files_Agile12v2016_RATER:
    filePath = "%s/%s" % ("Agile12v2016-RATER",file)
    fileInfo = filePath.split("_")
    rater = int(fileInfo[2][5:])
    iteration = int(fileInfo[3])
    
    data = getDataFromFcsv(filePath)
    version = data["version"]
    coordinateSystem = data["coordinateSystem"]
    nFiducials = len(data["data"])
    for index, row in data["data"].iterrows():
        toAdd = [rater, iteration, version, coordinateSystem]
        for ch in col_headers:
            val = row[ch].strip().upper()
            
            toAdd.append(val)
            #print(row["desc"])
            
        Agile12v2016_RATER.append(toAdd)
            
Agile12v2016_RATER_df = pd.DataFrame(Agile12v2016_RATER, 
                                     columns = ["rater","iteration","version","coordinateSystem","id","x","y","z","ow","ox","oy","oz","vis","sel","lock","label","desc","associatedNodeID"])

In [48]:
Agile12v2016_RATER_df

,rater,iteration,version,coordinateSystem,id,x,y,z,ow,ox,oy,oz,vis,sel,lock,label,desc,associatedNodeID
0,1,1,4.6,0,VTKMRMLMARKUPSFIDUCIALNODE_0,4.30596,5.50771,1.52776,6.38605E-316,3.85977E-316,6.16689E-316,0,1,1,1,1,AC,VTKMRMLSCALARVOLUMENODE1
1,1,1,4.6,0,VTKMRMLMARKUPSFIDUCIALNODE_1,3.03907,-19.726,-4.75643,6.38605E-316,3.85977E-316,6.16689E-316,4.94066E-324,1,1,1,2,PC,VTKMRMLSCALARVOLUMENODE1
2,1,1,4.6,0,VTKMRMLMARKUPSFIDUCIALNODE_3,2.38494,-29.3096,-16.1478,0,0,0,1,1,1,1,3,INFRACOLLICULAR SULCUS,VTKMRMLSCALARVOLUMENODE1
3,1,1,4.6,0,VTKMRMLMARKUPSFIDUCIALNODE_4,3.40552,-13.562,-20.0726,0,0,0,1,1,1,1,4,PMJ,VTKMRMLSCALARVOLUMENODE1
4,1,1,4.6,0,VTKMRMLMARKUPSFIDUCIALNODE_2,3.99437,-4.98765,-8.30469,6.38605E-316,3.85977E-316,6.16689E-316,9.88131E-324,1,1,1,5,SUPERIOR INTERPEDUNCULAR FOSSA,VTKMRMLSCALARVOLUMENODE1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000,8,4,4.6,0,VTKMRMLMARKUPSFIDUCIALNODE_30,-11.8232,-37.1502,-5.13688,0,0,0,1,1,1,1,28,L INDUSIUM GRISEUM ORIGIN,VTKMRMLSCALARVOLUMENODE1
1001,8,4,4.6,0,VTKMRMLMARKUPSFIDUCIALNODE_31,17.8605,-72.4159,-16.6626,0,0,0,1,1,1,1,29,R VENTRAL OCCIPITAL HORN,VTKMRMLSCALARVOLUMENODE1
1002,8,4,4.6,0,VTKMRMLMARKUPSFIDUCIALNODE_32,-17.435,-72.4807,-17.471,0,0,0,1,1,1,1,30,L VENTRAL OCCIPITAL HORN,VTKMRMLSCALARVOLUMENODE1
1003,8,4,4.6,0,VTKMRMLMARKUPSFIDUCIALNODE_33,17.1142,20.8435,0.857424,0,0,0,1,1,1,1,31,R OLFACTORY SULCAL FUNDUS,VTKMRMLSCALARVOLUMENODE1


In [73]:
#mean fiducial locations; by labels rather than desc as people use different descriptions...

mean_fiducials = []
for fid in range(1,33):
    
    fid_subset = Agile12v2016_RATER_df.loc[Agile12v2016_RATER_df["label"]==str(fid)]
    xVals = [float(x) for x in fid_subset["x"].tolist()]
    yVals = [float(y) for y in fid_subset["y"].tolist()]
    zVals = [float(z) for z in fid_subset["z"].tolist()]
    
    meanX = np.mean(xVals)
    meanY = np.mean(yVals)
    meanZ = np.mean(zVals)
    
    mean_fiducials.append([fid,meanX,meanY,meanZ])

mean_fiducials_df = pd.DataFrame(mean_fiducials,
                                columns = ["label","meanX","meanY","meanZ"])

In [74]:
print(mean_fiducials_df)

    label      meanX      meanY      meanZ
0       1   4.149391   5.678822   1.479532
1       2   3.230568 -19.906413  -4.875663
2       3   2.796073 -28.609881 -16.037834
3       4   3.421191 -13.232969 -20.471747
4       5   3.735649  -7.009849  -8.739796
5       6  15.007181 -18.928950 -11.421050
6       7  -8.988745 -18.226706 -11.611441
7       8  12.951063 -19.957878 -22.048188
8       9  -6.580120 -18.918334 -21.962928
9      10   1.901720 -45.484784  -8.999917
10     11   4.135242  -0.979683 -10.888862
11     12   6.271458  -1.114399 -10.123704
12     13   2.044852  -1.049872 -10.112024
13     14   2.712878 -25.869232  -4.811976
14     15  18.599119  -1.432479  27.078113
15     16 -10.379794  -0.498532  26.874461
16     17  19.451762 -25.229084  20.686581
17     18 -13.074877 -25.877752  20.187571
18     19   5.070870  28.582313  18.003632
19     20   2.778037 -33.272242  -1.486621
20     21  34.589332   4.674916 -19.132958
21     22 -26.067868   5.612301 -18.543352
22     23  

In [93]:
#get the rater differences from the mean 
diffs = []
for fid in range(1,33):
    fid_subset = Agile12v2016_RATER_df.loc[Agile12v2016_RATER_df["label"]==str(fid)]
    for key, value in fid_subset.iterrows():
        toAdd = [value["rater"],value["iteration"],value["label"]]
        
        means = mean_fiducials_df.loc[mean_fiducials_df["label"]==fid]
        
        meanX = float(means["meanX"])
        meanY = float(means["meanY"])
        meanZ = float(means["meanZ"])
        
        dist = math.sqrt((meanX-float(value["x"]))**2 + 
                     (meanY-float(value["y"]))**2 +
                     (meanZ-float(value["z"]))**2)
        
        toAdd.append(dist)
 
        diffs.append(toAdd)

diff_fiducials_df = pd.DataFrame(diffs,
                                columns = ["rater","iteration","label","dist"])

In [91]:
#calculating the mean and standard deviation of the differences 
AFLEs = []
for fid in range(1,33):
    fid_subset = diff_fiducials_df.loc[diff_fiducials_df["label"]==str(fid)]
    
    dist = fid_subset["dist"].tolist()
    
    toAdd = [fid,len(valuesX),len(valuesY),len(valuesZ)]
    
    meanDiff = np.mean(valuesX)
    stddevDiffZ = np.std(valuesZ)
    
    stats = [meanDiffX,meanDiffY,meanDiffZ,stddevDiffX,stddevDiffY,stddevDiffZ]
    
    toAdd = np.concatenate((toAdd, stats))
    
    AFLEs.append(toAdd)
    
AFLEs_df = pd.DataFrame(AFLEs,
                        columns = ["label","nX","nY","nZ","xMean","yMean","zMean","xSD","ySD","zSD"])    
    

In [92]:
AFLEs_df 

,label,nX,nY,nZ,xMean,yMean,zMean,xSD,ySD,zSD
0,1.0,32.0,32.0,32.0,2.359224e-16,3.330669e-16,-1.318390e-16,0.221161,0.149594,0.252515
1,2.0,32.0,32.0,32.0,-2.775558e-17,1.887379e-15,-1.665335e-16,0.210559,0.162996,0.279231
2,3.0,32.0,32.0,32.0,-2.775558e-16,3.330669e-16,2.664535e-15,0.361174,0.663061,1.097643
3,4.0,32.0,32.0,32.0,4.996004e-16,-3.330669e-16,9.992007e-16,0.263314,0.700141,0.575663
4,5.0,32.0,32.0,32.0,-2.636780e-16,-2.220446e-16,1.249001e-15,0.218437,1.195012,0.450819
5,6.0,32.0,32.0,32.0,-2.220446e-15,1.110223e-16,-6.661338e-16,0.571947,0.427072,0.528271
6,7.0,32.0,32.0,32.0,-1.221245e-15,-4.440892e-16,1.110223e-16,0.677901,0.535490,0.676453
7,8.0,32.0,32.0,32.0,-6.661338e-16,2.220446e-16,9.992007e-16,0.337425,0.552057,1.799427
8,9.0,32.0,32.0,32.0,-8.326673e-17,6.661338e-16,6.661338e-16,0.350818,0.514607,1.657442
9,10.0,32.0,32.0,32.0,5.551115e-17,3.774758e-15,2.775558e-16,0.613962,1.012637,0.429690
